# Linear Regression with PyTorch

##### Part 2 of "PyTorch: Zero to GANs"

we'll discuss one of the foundational algorithms of machine learning in this post: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall and humidity (input variables or features) in a region. Here's the training data:

![Dataset](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

`yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1`
`yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2`

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![Graph](https://i.imgur.com/4DJ9f8X.png)

The learning part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called _gradient descent_.

In [1]:
# Uncomment the command below if Numpy or PyTorch is not installed
# !conda install numpy pytorch cpuonly -c pytorch -y

In [2]:
import numpy as np
import torch

## Training Data

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [4]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [5]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [6]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.6734,  1.5077, -0.4217],
        [-1.4686, -0.4865,  0.1960]], requires_grad=True)
tensor([ 0.8478, -0.7982], requires_grad=True)


Our model is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![Operation](https://i.imgur.com/WGXLFvA.png)

Lets define the Model

In [7]:
def model(x):
    return x @ w.t() + b    # @ == *, .t() == Transpose

In [8]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  34.5719, -132.1724],
        [  45.2570, -164.7070],
        [ 119.8354, -182.3878],
        [ -18.6128, -164.2622],
        [  69.6043, -135.1133]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [9]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to just work.

## Loss Function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

- Calculate the difference between the two matrices (preds and targets).
- Square all elements of the difference matrix to remove negative values.
- Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error**(MSE).

In [10]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(32052.5508, grad_fn=<DivBackward0>)


## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [12]:
# Compute gradients
loss.backward()

In [13]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.6734,  1.5077, -0.4217],
        [-1.4686, -0.4865,  0.1960]], requires_grad=True)
tensor([[ -2238.2007,  -1884.2939,  -1400.1772],
        [-20887.8457, -22447.7793, -13845.1572]])


Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [15]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  34.5719, -132.1724],
        [  45.2570, -164.7070],
        [ 119.8354, -182.3878],
        [ -18.6128, -164.2622],
        [  69.6043, -135.1133]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.

In [16]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(32052.5508, grad_fn=<DivBackward0>)


In [17]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -2238.2007,  -1884.2939,  -1400.1772],
        [-20887.8457, -22447.7793, -13845.1572]])
tensor([ -26.0689, -247.7285])


Finally, we update the weights and biases using the gradients computed above.

In [18]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

- We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 

- We multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm. 

- After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.


Let's take a look at the new weights and biases.

In [19]:
print(w)
print(b)

tensor([[-0.6511,  1.5266, -0.4077],
        [-1.2597, -0.2620,  0.3345]], requires_grad=True)
tensor([ 0.8480, -0.7957], requires_grad=True)


In [20]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(21650.7383, grad_fn=<DivBackward0>)


# Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [21]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(80.7164, grad_fn=<DivBackward0>)


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets.

In [23]:
# Predictions
preds

tensor([[ 55.9137,  70.1726],
        [ 73.0355, 101.1559],
        [141.5663, 132.0707],
        [ 12.1424,  37.0435],
        [ 91.3494, 119.8116]], grad_fn=<AddBackward0>)

In [24]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs.

**--------------------------------------------------------------------------------------------------------------------------------**

# Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [25]:
import torch.nn as nn

In [26]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [27]:
from torch.utils.data import TensorDataset

In [28]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [29]:
from torch.utils.data import DataLoader

In [30]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [31]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[69., 96., 70.],
        [91., 88., 64.],
        [73., 67., 43.],
        [91., 88., 64.],
        [73., 67., 43.]])
tensor([[103., 119.],
        [ 81., 101.],
        [ 56.,  70.],
        [ 81., 101.],
        [ 56.,  70.]])


## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [32]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2629, -0.5562, -0.3641],
        [-0.2610,  0.3827,  0.0859]], requires_grad=True)
Parameter containing:
tensor([0.4870, 0.3736], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [33]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2629, -0.5562, -0.3641],
         [-0.2610,  0.3827,  0.0859]], requires_grad=True),
 Parameter containing:
 tensor([0.4870, 0.3736], requires_grad=True)]

In [34]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -71.6263,   10.6570],
        [ -95.6847,   15.8006],
        [-118.0348,   33.9354],
        [ -63.7160,   -6.6145],
        [ -96.5356,   25.1209],
        [ -71.6263,   10.6570],
        [ -95.6847,   15.8006],
        [-118.0348,   33.9354],
        [ -63.7160,   -6.6145],
        [ -96.5356,   25.1209],
        [ -71.6263,   10.6570],
        [ -95.6847,   15.8006],
        [-118.0348,   33.9354],
        [ -63.7160,   -6.6145],
        [ -96.5356,   25.1209]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [35]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [36]:
# Define loss function
loss_fn = F.mse_loss

In [37]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(18216.3027, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [38]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD`, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

In [39]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update, and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [40]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 317.1266
Epoch [20/100], Loss: 540.1069
Epoch [30/100], Loss: 290.4594
Epoch [40/100], Loss: 83.6784
Epoch [50/100], Loss: 84.8947
Epoch [60/100], Loss: 87.7578
Epoch [70/100], Loss: 73.7350
Epoch [80/100], Loss: 21.9793
Epoch [90/100], Loss: 21.6852
Epoch [100/100], Loss: 38.6523


In [41]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.8040,  71.2080],
        [ 80.6559,  97.8834],
        [119.5684, 138.1159],
        [ 29.8763,  41.3229],
        [ 94.1880, 111.8313],
        [ 58.8040,  71.2080],
        [ 80.6559,  97.8834],
        [119.5684, 138.1159],
        [ 29.8763,  41.3229],
        [ 94.1880, 111.8313],
        [ 58.8040,  71.2080],
        [ 80.6559,  97.8834],
        [119.5684, 138.1159],
        [ 29.8763,  41.3229],
        [ 94.1880, 111.8313]], grad_fn=<AddmmBackward>)

In [42]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Credit: https://jovian.ml/aakashns/02-linear-regression<br>
**THE END**